In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')

from stemplot import *

In [2]:
from stempy.io import *
from stempy.clustering import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

In [3]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CrWSe2_total 8 datasets\CrWSe2_20211028'+'\\'

# must change this
num = 15

file_name1 = dp + 'ADF1_{}.dm4'.format(num)
file_name2 = dp + 'ADF2_{}.dm4'.format(num)

# load image and normalize
img1 = load_image(file_name1)
img1 = normalize_image(img1, 0, 1)

img2 = load_image(file_name2)
img2 = normalize_image(img2, 0, 1) 

In [4]:
img1 = remove_bg(img1, 7)
img2 = remove_bg(img2, 7)

img1 = normalize_image(img1, 0, 1)
img2 = normalize_image(img2, 0, 1)

In [5]:
#plot_compare([img1, img2])

In [6]:
get_tmd_bond(img1), get_tmd_bond(img2)

(6, 6)

## denoising

In [7]:
n_components = 32
patch_size = 32
extraction_step = 2
imgf = denoise_svd(img2, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.18s.
Singular value decomposition...
done in 1.85s.
Reconstructing...
done in 1.04s.


In [8]:
imshow(imgf)

In [9]:
threshold = 0.1
pts = local_max(imgf, min_distance=1, threshold=threshold)

In [10]:
size = 23
kp1 = KeyPoints(pts, img1, size)
ps1 = kp1.extract_patches(size) 

kp2 = KeyPoints(pts, img2, size)
ps2 = kp2.extract_patches(size) 

In [11]:
#np.save('pts_Cr_15.npy', kp1.pts)

In [12]:
imshow(ps2)

In [13]:
zps = ZPs(n_max=12, size=ps1.shape[1])
zps.fit(ps1)
X1 = zps.moments
zps.fit(ps2)
X2 = zps.moments

X = np.hstack([X1, X2])

In [14]:
plot_pca(X, 2)

In [15]:
lbs = gmm_lbs(X, 2, n_init=1)

In [16]:
plot_pca(X, 2, lbs=lbs)

In [17]:
X11 = X[lbs==0]
X22 = X[lbs==1]

In [18]:
plot_pca(X11, 2, s=3)

In [19]:
if ps1[lbs==0].mean(axis=0)[size//2, size//2] > ps1[lbs==1].mean(axis=0)[size//2, size//2]:
    lbs_x = 1
    lbs_m = 0
else:
    lbs_x = 0
    lbs_m = 1

print(lbs_x)

0


In [20]:
X_x = np.hstack([X1.select([0])[lbs==lbs_x], X2.select([0])[lbs==lbs_x]])
X_m = np.hstack([X1.select([0])[lbs==lbs_m], X2.select([0])[lbs==lbs_m]])
plot_pca(X_x, 2, s=3)

In [25]:
l= interactive_clusters(pca(X_x), ps2[lbs==lbs_x], clip=False, s=3) 

One cluster has been selected.


In [31]:
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

davies_bouldin_score(X_x, l.lbs+1)
calinski_harabasz_score(X_x, l.lbs+1)

292.9429794539119

In [33]:
lbsx = gmm_lbs(X_x, 2, tol=1e-6)
#lbsm = gmm_lbs(X_m, 2)

In [39]:
plot_pca(X_m, 2, lbs=lbsm)

In [40]:
lbsx = gmm_lbs(X_x, 2, tol=1e-6)
plot_pca(X_x, 2, lbs=lbsx)

In [42]:
plot_pca(X_x, 2, lbs=lbsx)

In [43]:
np.save('X.npy', X_x)

In [44]:
np.save('ps.npy', ps2[lbs==lbs_x])

In [37]:
davies_bouldin_score(X_x, lbsx)

2.7828119170722956

In [42]:
num_negative_samples = 5
if lbs_x == 0:
    fg = ForceGraph8(X = X11,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=num_negative_samples,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X11)
else:
    fg = ForceGraph8(X = X22,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=num_negative_samples,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X22)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [45]:
l= interactive_clusters(xy, img2, kp1.pts[lbs==lbs_x], ps2[lbs==lbs_x], clip=False, s=1) 

One cluster has been selected.


In [46]:
plot_pca(X_x, 2, lbs=l.lbs+1)

In [47]:
def combine_lbs(lbs, lbsx, lbsm, lbs_x, lbs_m):
    lbs_ = lbs.copy()
    lbsx = lbsx + 3
    lbsm = lbsm + 5
    lbs_[lbs_==lbs_x] = lbsx 
    lbs_[lbs_==lbs_m] = lbsm
    return reorder_lbs(lbs_)

def reorder_lbs(lbs):
    unique, counts = np.unique(lbs, return_counts=True)
    cnts_order = np.argsort(counts)[::-1]
    unique = unique[cnts_order]
    order_dict = dict(zip(unique, np.arange(len(counts))))
    lbs = np.vectorize(order_dict.get)(lbs)
    return lbs

In [48]:
lbs_ = combine_lbs(lbs, l.lbs, lbsm, lbs_x, lbs_m)

In [52]:
np.save('lbs_Cr_15.npy', lbs_)

In [49]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img2, cmap='gray')
ax.scatter(kp1.pts[:, 0], kp1.pts[:, 1], color=colors_from_lbs(lbs_), s=10) 

In [225]:
from sklearn.neighbors import NearestNeighbors

def get_pairs_motif(pts, lbs, radius=10):
    pts1 = pts[lbs==2]
    pts2 = pts[lbs==3]
    nbrs = NearestNeighbors(radius=radius, algorithm='ball_tree').fit(pts1)
    d, ind = nbrs.radius_neighbors(pts2)  
    print(d, ind)

In [239]:
x, y = 190, 240
pair_motif5 = img1[y-11:y+12, x-11:x+12]
pair_motif6 = img2[y-11:y+12, x-11:x+12]

In [240]:
x, y = 270, 194
pair_motif1 = img1[y-11:y+12, x-11:x+12]
pair_motif2 = img2[y-11:y+12, x-11:x+12]
x, y = 107, 168
pair_motif3 = img1[y-11:y+12, x-11:x+12]
pair_motif4 = img2[y-11:y+12, x-11:x+12]
x, y = 190, 240
pair_motif5 = img1[y-11:y+12, x-11:x+12]
pair_motif6 = img2[y-11:y+12, x-11:x+12]
np.save('ADF1_pair.npy', np.array([pair_motif1, pair_motif3,pair_motif5]).mean(axis=0))
np.save('ADF2_pair.npy', np.array([pair_motif2, pair_motif4,pair_motif6]).mean(axis=0))

In [230]:
plot_compare([pair_motif1, pair_motif2])

In [208]:
fig, axes = plt.subplots(1, 4, figsize=(7.2, 7.2))
ax1, ax2, ax3, ax4 = axes
plot_image(ax1, ps2[lbs_==0].mean(axis=0), cmap=color_palette('C0'), vmin=0, vmax=1)
plot_image(ax2, ps2[lbs_==1].mean(axis=0), cmap=color_palette('C1'), vmin=0, vmax=1)
plot_image(ax3, ps2[lbs_==2].mean(axis=0), cmap=color_palette('C2'), vmin=0, vmax=1)
plot_image(ax4, ps2[lbs_==3].mean(axis=0), cmap=color_palette('C3'), vmin=0, vmax=1)

In [211]:
fig, axes = plt.subplots(1, 4, figsize=(7.2, 7.2))
ax1, ax2, ax3, ax4 = axes
plot_image(ax1, ps1[lbs_==0].mean(axis=0), cmap=color_palette('C0'), vmin=0, vmax=1)
plot_image(ax2, ps1[lbs_==1].mean(axis=0), cmap=color_palette('C1'), vmin=0, vmax=1)
plot_image(ax3, ps1[lbs_==2].mean(axis=0), cmap=color_palette('C2'), vmin=0, vmax=1)
plot_image(ax4, ps1[lbs_==3].mean(axis=0), cmap=color_palette('C3'), vmin=0, vmax=1)

In [212]:
np.save('ADF1_W.npy', ps1[lbs_==0].mean(axis=0))
np.save('ADF1_Se.npy', ps1[lbs_==1].mean(axis=0))
np.save('ADF1_Vse.npy', ps1[lbs_==2].mean(axis=0))
np.save('ADF1_Cr.npy', ps1[lbs_==3].mean(axis=0))

In [213]:
np.save('ADF2_W.npy', ps2[lbs_==0].mean(axis=0))
np.save('ADF2_Se.npy', ps2[lbs_==1].mean(axis=0))
np.save('ADF2_Vse.npy', ps2[lbs_==2].mean(axis=0))
np.save('ADF2_Cr.npy', ps2[lbs_==3].mean(axis=0))

In [98]:
num_negative_samples = 5
if lbs_x == 0:
    fg = ForceGraph8(X = X11,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=num_negative_samples,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X11)
else:
    fg = ForceGraph8(X = X22,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=num_negative_samples,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X22)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [99]:
l= interactive_clusters(xy, img2, kp1.pts[lbs==lbs_x], ps2[lbs==lbs_x], clip=False, s=1) 

One cluster has been selected.


In [100]:
len(l.ind)

93

## get dopant by intensity

In [84]:
pts_m = kp1.pts[lbs==lbs_m]
s = 1
I = np.array([img2[y-s:y+s+1, x-s:x+s+1].mean() for (x, y) in pts_m])
II = np.array([np.array([img2[y-s:y+s+1, x-s:x+s+1].mean() for (x, y) in pts_m]) for s in [1,2,3]]).T

In [91]:
lbs_dopants = gmm_lbs(II, 2, type='tied')
np.unique(lbs_dopants, return_counts=True)

(array([0, 1], dtype=int64), array([1716,   23], dtype=int64))

In [92]:
plot_pca(II, 2, lbs=lbs_dopants)

In [93]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img2, cmap='gray')
ax.scatter(kp1.pts[lbs==lbs_m][:, 0], kp1.pts[lbs==lbs_m][:, 1], c=lbs_dopants, s=30) 

In [95]:
plot_pca(X[lbs==lbs_m][lbs_dopants==1])

In [96]:
imshow(ps2[lbs==lbs_m][lbs_dopants==1])

## get pairs FR

In [51]:
lbs_selected = 0
if lbs_selected == 0:
    fg = ForceGraph8(X = X11[:, 1:],
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=5,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X11)
else:
    fg = ForceGraph8(X = X22,
    n_neighbors=10,
    init_mode = 'pca',
    num_iterations = 100,
    num_negative_samples=5,
    force_params1=(0, 2, 1, 1),
    force_params2=(2, 5, 5, 1),
    divide=0.5)
    
    from time import time
    t0 = time()
    xy = fg.fit_transform(X22)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [52]:
l= interactive_clusters(xy, img2, kp1.pts[lbs==lbs_selected], ps2[lbs==lbs_selected], clip=False, s=1) 

In [31]:
pts_vac = kp1.pts[lbs==lbs_selected][l.ind]

In [38]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img1, cmap='gray')
ax.scatter(pts_vac[:, 0], pts_vac[:, 1], color='r', s=10) 

In [37]:
np.save('pts_vac.npy', pts_vac)

In [445]:
from sklearn.manifold import TSNE
Xt = TSNE(n_components=2, n_iter=2000).fit_transform(X22)

In [531]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xt[:, 0], Xt[:, 1], s=3)

In [532]:
l= interactive_clusters(Xt, img, kp1.pts[lbs==1], ps2[lbs==1], clip=False, s=1) 

In [450]:
motif1 = ps1[lbs==1][l.ind].mean(axis=0)
motif2 = ps2[lbs==1][l.ind].mean(axis=0)
imshow(motif1)

In [518]:
from skimage.util import random_noise

shape = (X22.shape[0], size, size)
ps3 = np.broadcast_to(motif1, shape)
ps4 = np.broadcast_to(motif2, shape)
ps3 = random_noise(ps3, var=0.01)
ps4 = random_noise(ps4, var=0.01)


zps.fit(ps3)
X3 = zps.moments
zps.fit(ps3)
X4 = zps.moments

X34 = np.hstack([X3, X4])
Xnew = np.vstack([X22, X34])

In [528]:
lbsnew = kmeans_lbs(Xnew, 2)

C:\Users\Ted\anaconda3\envs\jiadong\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [529]:
plot_pca(Xnew, lbs=lbsnew)

In [530]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xtnew[:, 0], Xtnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [520]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xnew[:, 0], Xnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [486]:
Xtnew = TSNE(n_components=2, n_iter=2000).fit_transform(Xnew)

In [509]:
lbsnew = [0]*(X22.shape[0])+[1]*(X22.shape[0])

In [521]:
plot_pca(Xnew, lbs=lbsnew)

In [523]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(Xtnew[:, 0], Xtnew[:, 1], s=3, color=colors_from_lbs(lbsnew))

In [292]:
motif1 = X[lbs==1].mean(axis=0)

In [293]:
from skimage.metrics import structural_similarity
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
kk = np.array([1-cosine(e, motif1) for e in X[lbs==1]])

In [294]:
plt.hist(kk, bins=200)

(array([  1.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   1.,
          1.,   0.,   0.,   0.,   1.,   1.,   0.,   2.,   0.,   0.,   2.,
          0.,   2.,   1.,   2.,   3.,   1.,   1.,   2.,   3.,   4.,   2.,
          1.,   3.,   1.,   2.,   2.,   3.,   3.,   1.,   1.,   3.,   5.,
          6.,   6.,   5.,   9.,   7., 

In [128]:
hh = ps_[lbs==0][:, 14-1:14+2, 14-1:14+2].mean(axis=(1,2))

In [129]:
plt.hist(hh, bins=100)

(array([  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          3.,   0.,   2.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,
          0.,   0.,   1.,   2.,   1.,   2.,   1.,   4.,   2.,  10.,  13.,
         14.,  22.,  25.,  57.,  33.,  76.,  82., 109., 101., 134., 136.,
        217., 209., 239., 267., 285., 309., 298., 305., 328., 320., 267.,
        291., 245., 235., 184., 194., 132., 114.,  94.,  79.,  63.,  42.,
         45.,  35.,  29.,  29.,  14.,  12.,   4.,  10.,   7.,   4.,   0.,
          2.,   2.,   3.,   5.,   1.,   0.,   1.,   1.,   1.,   0.,   0.,
          1.]),
 array([0.04777327, 0.05260071, 0.05742816, 0.06225561, 0.06708306,
        0.07191051, 0.07673796, 0.0815654 , 0.08639285, 0.0912203 ,
        0.09604775, 0.1008752 , 0.10570264, 0.11053009, 0.11535754,
        0.12018499, 0.12501244, 0.12983989, 0.13466733, 0.13949478,
        0.14432223, 0.14914968, 0.15397713, 0.

In [125]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=5,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [336]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False) 

In [15]:
l.ind

array([  24,   95,  133,  157,  194,  250,  310,  447,  597,  612,  626,
        671,  690,  851,  877,  882,  913, 1041, 1109, 1112, 1120, 1351,
       1365, 1407, 1437, 1470, 1534, 1539, 1697, 1716, 1793, 1822, 2018,
       2064, 2167, 2213, 2386, 2407, 2416, 2445, 2499, 2560, 2719, 2829,
       2915, 2920, 2930, 2942, 2944, 2957, 3025, 3041, 3100, 3206, 3229,
       3243, 3359, 3367, 3375, 3393, 3523, 3531, 3562, 3594, 3676, 3703,
       3892, 4037, 4263, 4378, 4391, 4440, 4449, 4478, 4485, 4576, 4586,
       4590, 4642, 4665, 4727, 4740, 4761, 4772, 4809, 4854, 4863, 4897,
       4910, 4925, 4975, 4982, 5030, 5055, 5077, 5398, 5413, 5415, 5580,
       5628, 5690, 5699, 5882, 5883, 5891, 5970, 5972, 6031, 6111, 6154,
       6167, 6248, 6264, 6349, 6387, 6428, 6432, 6438, 6541, 6554, 6558,
       6562, 6565, 6651, 6674, 6693, 6934, 7006, 7061, 7142, 7144, 7194,
       7334, 7348, 7355, 7421, 7452, 7603, 7682, 7697, 7745, 7846],
      dtype=int64)

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)